[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/juansensio/axr/blob/master/axr/00_intro.ipynb)

# Introducción

## Ejemplo de aplicación: tres en raya



Con el objetivo de ilustrar la idea general del axr vamos a considerar un ejemplo en detalle: el juego del tres en raya. En este juego, dos jugadores se turnan para dibujar una X o una O en un tablero con 3x3 posiciones. El primer jugador en conseguir dibujar tres figuras en una línea horizontal, vertical o diagonal, gana. Nuestro objetivo es conseguir un agente que sea capaz de ganar siempre a este juego.

![](https://camo.githubusercontent.com/9b5f16d0451a8b6faf1ec45a6afbe22f55bbe1f9/68747470733a2f2f7468756d62732e6766796361742e636f6d2f506f697365644772697070696e67466f782d736d616c6c2e676966)

Para resolver este juego con axr, en primer lugar definimos una tabla de números, uno por cada posible estado del juego. Cada numero respresentará la probabilidad de ganar el juego desde ese estado, el *valor* del estado. Así pues, la tabla sería la *función de valor*. Un estado $A$ es considerado mejor que un estado $B$ si el valor estimado de la probabilidad de ganar el juego desde $A$ es mayor que desde $B$. Si jugásemos con las Xs, todos los estados con tres X en raya tenría un valor de 1, ya que hemos ganado el juego. De la misma manera, cualquier estado con tres Os en raya tendría un valor de 0, hemos perdido. Para la inicialización de la tabla, podemos establecer el resto de valores en 0.5 (50% de posibilidades de ganar).

Nuestro agente jugará muchas partidas contra un oponente (que puede ser otro agente). En cada turno evaluamos los estados que resultarían de cada posible movimiento (posiciones no ocupadas) y elegimos aquella con un mayor *valor*. Ocasionalmente, elegiremos una acción aleatoria con el objetivo de explorar nuevos movimientos.
        
Mientras el agente va jugando, tendremos que actualizar la función de valor. Para ello, después de cada movimiento, cambiaremos el valor del estado del que venimos para que se acerque al valor del estado actual.

$$
V(S_t) \leftarrow V(S_t) + \alpha [V(S_{t+1}) - V(S_t)]
$$
      
donde $S_t$ denote el estado del que venimos, $S_{t+1}$ es el nuevo estado después del movimiento, $V(S_t)$ es el valor del estado $S_t$ y $\alpha$ es el ratio de aprendizaje.

In [1]:
import numpy as np


class Board():
    def __init__(self):
        self.state = np.zeros((3,3))

    def valid_moves(self):
        return [(i, j) for j in range(3) for i in range(3) if self.state[i, j] == 0]

    def update(self, symbol, row, col):
        if self.state[row, col] == 0:
            self.state[row, col] = symbol
        else:
            raise ValueError ("movimiento ilegal !")

    def is_game_over(self):
        # comprobar filas y columnas
        if (self.state.sum(axis=0) == 3).sum() >= 1 or (self.state.sum(axis=1) == 3).sum() >= 1:
            return 1
        if (self.state.sum(axis=0) == -3).sum() >= 1 or (self.state.sum(axis=1) == -3).sum() >= 1:
            return -1 
        # comprobar diagonales
        diag_sums = [
            sum([self.state[i, i] for i in range(3)]),
            sum([self.state[i, 3 - i - 1] for i in range(3)]),
        ]
        if diag_sums[0] == 3 or diag_sums[1] == 3:
            return 1
        if diag_sums[0] == -3 or diag_sums[1] == -3:
            return -1        
        # empate
        if len(self.valid_moves()) == 0:
            return 0
        # seguir jugando
        return None

    def reset(self):
        self.state = np.zeros((3,3))


In [2]:
from tqdm import tqdm 

class Game():
    def __init__(self, player1, player2):
        player1.symbol = 1
        player2.symbol = -1
        self.players = [player1, player2]
        self.board = Board()

    def selfplay(self, rounds=100):
        wins = [0, 0]
        for i in tqdm(range(1, rounds + 1)):
            self.board.reset()
            for player in self.players:
                player.reset()
            game_over = False
            while not game_over:
                for player in self.players:
                    action = player.move(self.board)
                    self.board.update(player.symbol, action[0], action[1])
                    for player in self.players:
                        player.update(self.board)
                    if self.board.is_game_over() is not None:
                        game_over = True
                        break
            self.reward()
            for ix, player in enumerate(self.players):
                if self.board.is_game_over() == player.symbol: 
                    wins[ix] += 1
        return wins


    def reward(self):
        winner = self.board.is_game_over()
        if winner == 0: # empate
            for player in self.players:
                player.reward(0.5)
        else: # le damos 1 recompensa al jugador que gana
            for player in self.players:
                if winner == player.symbol:
                    player.reward(1)
                else:
                    player.reward(0)

In [3]:
class Agent():
    def __init__(self, alpha=0.5, prob_exp=0.5):
        self.value_function = {} # tabla con pares estado -> valor
        self.alpha = alpha         # learning rate
        self.positions = []       # guardamos todas las posiciones de la partida
        self.prob_exp = prob_exp   # probabilidad de explorar

    def reset(self):
        self.positions = []

    def move(self, board, explore=True):
        valid_moves = board.valid_moves()
        # exploracion
        if explore and np.random.uniform(0, 1) < self.prob_exp:
            # vamos a una posición aleatoria
            ix = np.random.choice(len(valid_moves))
            return valid_moves[ix]
        # explotacion
        # vamos a la posición con más valor
        max_value = -1000
        for row, col in valid_moves:
            next_board = board.state.copy()
            next_board[row, col] = self.symbol
            next_state = str(next_board.reshape(3*3))
            value = 0 if self.value_function.get(next_state) is None else self.value_function.get(next_state)
            if value >= max_value:
                max_value = value
                best_row, best_col = row, col
        return best_row, best_col

    def update(self, board):
        self.positions.append(str(board.state.reshape(3*3)))

    
    def reward(self, reward):
        # al final de la partida (cuando recibimos la recompensa)
        # iteramos por tods los estados actualizando su valor en la tabla
        for p in reversed(self.positions):
            if self.value_function.get(p) is None:
                self.value_function[p] = 0
            self.value_function[p] += self.alpha * (reward - self.value_function[p])
            reward = self.value_function[p]

In [6]:
agent1 = Agent(prob_exp=0.5)
agent2 = Agent()

game = Game(agent1, agent2)

game.selfplay(10000)

100%|██████████| 10000/10000 [00:48<00:00, 207.70it/s]


[5615, 2750]

In [7]:
import pandas as pd

funcion_de_valor = sorted(agent1.value_function.items(), key=lambda kv: kv[1], reverse=True)
tabla = pd.DataFrame({'estado': [x[0] for x in funcion_de_valor], 'valor': [x[1] for x in funcion_de_valor]})

tabla

,estado,valor
0,[ 1. 1. -1. 1. -1. -1. 1. -1. 1.],1.0
1,[-1. 0. 1. 0. -1. 1. 0. 0. 1.],1.0
2,[ 1. 1. 1. 1. -1. -1. -1. -1. 1.],1.0
3,[ 1. 0. 0. -1. 1. 0. -1. 0. 1.],1.0
4,[ 0. -1. 0. 0. -1. 0. 1. 1. 1.],1.0
...,...,...
4706,[ 1. 1. 0. 0. 1. -1. 1. -1. -1.],0.0
4707,[-1. 1. 1. 1. 0. 0. -1. -1. 0.],0.0
4708,[ 1. 1. -1. 0. -1. 1. 0. -1. 0.],0.0
4709,[-1. 1. 0. -1. 0. 1. -1. 1. 0.],0.0


In [8]:
import pickle

with open('agente.pickle', 'wb') as handle:
    pickle.dump(agent1.value_function, handle, protocol=pickle.HIGHEST_PROTOCOL)

Este ejemplo sirve para ilustrar algunas de las propiedades clave del axr. En primer lugar, aprender a través de la interacción con el entorno (en este caso el otro agente). En segundo lugar, tenemos un objetivo claro y el comportamiento correcto del agente requiere de planificación y predicción que tenga en cuenta los efectos futuros de sus acciones.

##  Resumen

El aprendizaje por refuerzo es una aproximación computacional a la comprensión y automatización del aprendizaje por objetivos y toma de decisiones. En esta aproximación, una agente aprende a través de la interacción directa con su entorno sin necesidad de supervisión explícita. Utiliza procesos de decisión de Markov para definir la interacción entre el agente y su entorno en términos de estados, acciones y recompensas. Los conceptos de valor y función de valor son la clave de muchos métodos de axr ya que representan una manera eficiente de búsqueda en el espacio de políticas.